In [ ]:
#Paqueterias
import re
import unicodedata
from nltk import ngrams
import pandas as pd
from fuzzywuzzy import fuzz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
#Carga de datos
from glob import glob
in_dir = '/content/drive/MyDrive/HackathonRIIAA2021/TextData'
files = glob(in_dir+'/*.txt')
notas=[]
for file in files:
  f=open(file,'r')
  texto=f.read()
  notas.append(texto)
  f.close()

In [6]:
# Quita acentos
def quitar_acentos(text):
  try:
    text = unicode(text, 'utf-8')
  except (TypeError, NameError): 
    pass
  text = unicodedata.normalize('NFD', text)
  text = text.encode('ascii', 'ignore')
  text = text.decode("utf-8")
  return str(text)

In [7]:
def encontrar(palabras, text,label):
  copia = text.lower()
  copia = re.sub(r'[^\w\s]','',copia)
  copia = quitar_acentos(copia)
  entidades=[]
  for palabra in palabras:
    if palabra in copia:
      entidades.append((palabra,label))
      break
    num=len(palabra.split(' '))+1
    grams = ngrams(copia.split(' '), num)
    for gram in grams:
      cadena=' '.join(gram)
      if fuzz.ratio(palabra,cadena)>90:
        entidades.append((cadena,label))
  return entidades         

In [ ]:
nombres = pd.read_csv('/content/drive/MyDrive/Datos - Hackathon JusticIA/civilservants.csv', error_bad_lines=False)
organizaciones = pd.read_csv('/content/drive/MyDrive/Datos - Hackathon JusticIA/organizations.csv', error_bad_lines=False)

In [ ]:
wiki=[]
nu=1
for nota in notas:
  wiki_=[] 
  wiki_.extend(encontrar(nombres['CIVILSERVANTS '], nota, 'PER'))
  wiki_.extend(encontrar(organizaciones['ORGANIZATIONS '], nota, 'ORG'))
  wiki.append((nota,{'entities':wiki_}))
  print(nu)
  nu=nu+1